# Execute the Graph-Based Spatial Cross-Validation experiments from ICMLA21 Paper

## 1 - Initialize libraries

In [26]:
import os
import pandas as pd
import geopandas as gpd
from weka.core import jvm
from pathlib import Path
from src import utils
from src.pipeline import Pipeline
from src.visualization.performance import VizMetrics
from src.visualization.dependence import VizDependence

## 2 - Initialize loggers

In [27]:
utils.initialize_coloredlog()
utils.initialize_rich_tracerback()
utils.initialize_logging()

## 3 - Initialize working path and enviromental variables

In [28]:
# Project path
project_dir = str(Path().resolve().parents[1])
# Load enviromental variables
env_var = utils.load_env_variables(project_dir)

## 4 - Set pipeline switchers, the default is to set True to all processes

In [29]:
# Set pipeline switchers
SWITCHERS = {
    "scv": False,
    "fs": True,
    "train": False,
    "predict": False,
    "evaluate": False,
}

# 4 - List all datasets

In [30]:
brazil_removed_datasets = ["Brazil_Election_2018_Sampled_dec0.3_prob0.1_removed_north",
                   "Brazil_Election_2018_Sampled_dec0.3_prob0.1_removed_northeast",
                   "Brazil_Election_2018_Sampled_dec0.3_prob0.1_removed_south",
                   "Brazil_Election_2018_Sampled_dec0.3_prob0.1_removed_southeast",
                   "Brazil_Election_2018_Sampled_dec0.3_prob0.1_removed_centerwest"]

brazil_datasets = ["Brazil_Election_2018_Sampled_dec0.3_prob0.1",
                   "Brazil_Election_2018_Sampled_dec0.3_prob0.2",
                   "Brazil_Election_2018_Sampled_dec0.3_prob0.3",
                   "Brazil_Election_2018_Sampled_dec0.3_prob0.4",
                   "Brazil_Election_2018_Sampled_dec0.3_prob0.5",
                   "Brazil_Election_2018_Sampled_dec0.3_prob0.6",
                   "Brazil_Election_2018_Sampled_dec0.3_prob0.7",
                   "Brazil_Election_2018_Sampled_dec0.3_prob0.8",
                   "Brazil_Election_2018_Sampled_dec0.3_prob0.9"]
    

brazil_geoeconomical_regions = ["Brazil_Election_2018_removed_AMAZONIA",
                                "Brazil_Election_2018_removed_NORDESTE",
                                "Brazil_Election_2018_removed_CENTRO_SUL"]

us_corn_datasets = ["US_Corn_Yield_2016_Removed_Northeast",
                    "US_Corn_Yield_2016_Removed_Southeast",
                    "US_Corn_Yield_2016_Removed_Midwest",
                    "US_Corn_Yield_2016_Removed_Southwest",
                    "US_Corn_Yield_2016_Removed_West"]
just_for_test = ["Brazil_Election_2018"]

## 6 - Runs the pipeline for each method
OBS: The results and files generated from the pipeline execution will be in the created folder Results in the data directory

### 6.1 Traditional SCV
OBS: We the the paramenter fast True so the semivariogram calculation step that can take 24h is skipped. We calculate the removing buffer by considering the 27 n-degree neighborhood as stated in the paper 

In [31]:
dataset_list = us_corn_datasets
fs_method = "All"
#ml_methods = ["KNN", "OLS", "Lasso", "Ridge", "ElasticNet", "DT", "LGBM", "RF", "MLP", "SVM"]
ml_methods = ["KNN"]

if fs_method == "CFS" and SWITCHERS["fs"]:
    jvm.start()
# Set paths
meshblocks_filename = "cb_2016_us_county_500k.shp"
meshblocks_idx = "GEOID"
data_idx = "INDEX"
env_var["root_path"] = "/exp/tpinho/Datasets/US_Corn_Yield_2016"
for dataset in dataset_list:
    data_path = os.path.join(env_var["root_path"], dataset, "data.csv")
    meshblocks_path = os.path.join(env_var["root_path"], dataset, "meshblocks", meshblocks_filename)
    # Load data
    data = pd.read_csv(data_path, index_col=data_idx, low_memory=False)
    try:
        data.drop(columns=["[GEO]_LATITUDE", "[GEO]_LONGITUDE"], inplace=True)
    except KeyError:
        pass
    meshblocks = gpd.read_file(meshblocks_path)
    # Set meshblocks  index
    meshblocks.set_index(meshblocks_idx, inplace=True)
    for ml_method in ml_methods:
        # Run pipeline
        TraditionalSCV = Pipeline(
            root_path=os.path.join(env_var["root_path"], dataset),
            data=data,
            meshblocks=meshblocks,
            index_col="INDEX",
            fold_col="INDEX_FOLDS",
            target_col="TARGET",
            scv_method="TraditionalSCV",
            fs_method=fs_method,
            ml_method=ml_method,
            switchers=SWITCHERS
        )

        print(f"Running the Traditional SCV approach for dataset: {dataset} ML Method = {ml_method}")
        TraditionalSCV.run()
if fs_method == "CFS" and SWITCHERS["fs"]:
    jvm.stop()

Running the Traditional SCV approach for dataset: US_Corn_Yield_2016_Removed_Northeast ML Method = KNN


Selecting Features: 100%|██████████| 31/31 [00:00<00:00, 200.09it/s]


Running the Traditional SCV approach for dataset: US_Corn_Yield_2016_Removed_Southeast ML Method = KNN


Selecting Features: 100%|██████████| 25/25 [00:00<00:00, 180.15it/s]


Running the Traditional SCV approach for dataset: US_Corn_Yield_2016_Removed_Midwest ML Method = KNN


Selecting Features: 100%|██████████| 24/24 [00:00<00:00, 180.82it/s]


Running the Traditional SCV approach for dataset: US_Corn_Yield_2016_Removed_Southwest ML Method = KNN


Selecting Features: 100%|██████████| 33/33 [00:00<00:00, 182.60it/s]


Running the Traditional SCV approach for dataset: US_Corn_Yield_2016_Removed_West ML Method = KNN


Selecting Features: 100%|██████████| 31/31 [00:00<00:00, 190.44it/s]
